In [1]:
import sys, time
sys.path.append("..")
from motion.motion_controller import LateralController
from carlasim.carla_client import CarlaClient
from carlasim.ego_car import EgoCar
from planner.stub_slam import StubSLAM
from planner.stub_global_planner import StubGlobalPlanner

import carla

import gi
gi.require_version('Gst', '1.0')
gi.require_version('GstRtspServer', '1.0')
from gi.repository import Gst, GLib
Gst.init(None)

/workspaces/crawler-poc/notebooks/../carlasim/video_streamer.py:1: PyGIWarning: Gst was imported without specifying a version first. Use gi.require_version('Gst', '1.0') before import to ensure that the right version gets loaded.
  from gi.repository import Gst


[]

In [2]:
StartPosition = ['Town07', -100, 0, 0, 0.0]
global_planner = StubGlobalPlanner()
global_planner.read_mission("./small_mission.dat")
client = CarlaClient(town=StartPosition[0])
ego = EgoCar(client)
ego.set_pose(StartPosition[1], StartPosition[2], StartPosition[3], StartPosition[4])
ego.set_power(0)
ego.set_steering(0)

In [3]:
from model.vehicle_pose import VehiclePose
def draw_pose(p: VehiclePose):
     world = client.get_world()
     world.debug.draw_string(carla.Location(p.x, p.y, 2), 'X', draw_shadow=False,
                                        color=carla.Color(r=0, g=0, b=255), life_time=12000.0,
                                        persistent_lines=True)

In [4]:
world = client.get_world()

for w in global_planner.get_all_poses():    
    world.debug.draw_string(carla.Location(w.x, w.y, 2), 'O', draw_shadow=False,
                                       color=carla.Color(r=255, g=0, b=0), life_time=12000.0,
                                       persistent_lines=True)

In [8]:
def set_sterr(angle: float) -> None:
    ego.set_steering(angle)
    pass

def invalid_path() -> None:
    print("motion has reached an invalid path")

slam = StubSLAM(ego)

ctrl = LateralController (
    vehicle_length=2,
    #odometer=lambda : ego.odometer.read(),
    odometer=lambda : 10,
    steering_actuator=lambda a : set_sterr(a),
    slam_find_current_pose=lambda : slam.estimate_ego_pose()
)

path = global_planner.get_all_poses()




In [84]:
from motion.path_position_finder import PathPositionFinder
finder = PathPositionFinder(path)
ego.set_pose(-100,0,0,0)
ego.set_steering(0)
time.sleep(0.5)
pos = 0

In [91]:
i = finder.find_next_pos(pos, slam.estimate_ego_pose())

if i > 0:
    pos = i
    
ctrl.set_reference_path(path[pos], path[pos+1])
ctrl.loop(2)
ego.set_brake(0)
ego.set_power(100)
time.sleep(1)
ego.set_power(0)
ego.set_brake(1)

path_heading = -0.5345248312874611, vehicle heading: -5.540775299072266
[2] new heading computed: 1.9452060634203885
[2] ke = -0.5347619146223249, he = 0.08737555384235116


In [12]:
ego.set_pose(-100,0,0,2.6510994423327623)
draw_pose(path[4])
draw_pose(path[5])

In [ ]:
ego.set_brake(0)
ego.set_power(100)
time.sleep(1)
ego.set_brake(1.0)
ctrl.loop(1)


In [ ]:
ctrl.set_reference_path(path[5], path[6])

In [ ]:
ego.set_pose(-7,0,0,-27.51)

In [ ]:
print(path[4])

In [ ]:
print(path[5])